In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight

In [2]:
# Load the dataset
df = pd.read_csv("combined_output.csv", on_bad_lines='skip')

In [6]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,'5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d...,'kabargayo.com,'Kredit sepeda motor bisa terbayar jika Anda m...,'https://www.kabargayo.com/2024/09/19/kredit-s...,"'Jakarta, VIVA – Pembayaran kredit sepeda moto...",19/09/2024 22.32,'id,19/09/2024 22.32,'Aldi Hadad,'https://i1.wp.com/thumb.viva.co.id/media/fron...,NaN,'positive,NaN,"'Hari Pembayaran Berbayar atau Harcilnas 2024,...",NaN,5250000,Adira,NaN,NaN
1,'e1e3f8d68b58568e8217b7562d48de634fceb0d837135...,'viva.co.id,'Kredit Motor Bisa Lunas Jika Bayar Cicilan Te...,'https://www.viva.co.id/otomotif/tips/1753596-...,"'Jakarta, VIVA – Cicilan kredit motor yang ser...",19/09/2024 22.30,'id,19/09/2024 22.30,'Krisna Wicaksono,'https://thumb.viva.co.id/media/frontend/thumb...,NaN,'positive,NaN,"'Harinya Cicilan Lunas,2024,PT Adira Dinamika ...",NaN,5250000,Adira,NaN,NaN
2,'dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf...,'kabarmegapolitan.pikiran-rakyat.com,'Adira Finance Umumkan Pemenang HARCILNAS 2024...,'https://kabarmegapolitan.pikiran-rakyat.com/b...,'KABARMEGAPOLITAN.com - PT Adira Dinamika Mult...,19/09/2024 21.45,'id,19/09/2024 21.45,'Yuliansyah,'https://assets.pikiran-rakyat.com/www/network...,NaN,'positive,'HARCILNAS merupakan wujud apresiasi kami kepa...,"'PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 ...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,'56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b...,'banggairaya.id,"'Dapatkan Promo Menarik, Yamaha Prima Motor Ra...",'https://banggairaya.id/dapatkan-promo-menarik...,'BANGGAI RAYA- Yamaha Prima Motor ramaikan pam...,19/09/2024 19.45,'id,19/09/2024 19.45,'Chikal Connect,'https://i0.wp.com/banggairaya.id/wp-content/u...,NaN,'neutral,NaN,"'RAYA- Yamaha Prima Motor,Banggai Goverment Ex...",NaN,5250000,Adira,NaN,NaN
4,'1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721...,'jakarta.tribunnews.com,"'Sindikat Penipuan Leasing, Satu Bulan Ajukan ...",'https://jakarta.tribunnews.com/2024/09/19/sin...,'Laporan wartawan TribunJakarta.com Yusuf Bach...,19/09/2024 18.43,'id,19/09/2024 18.43,'Yusuf Bachtiar,'https://asset-2.tstatic.net/jakarta/foto/bank...,NaN,'neutral,'Pelaku ini melakukan pembiayaan pembelian ken...,"'Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [3]:
# Clean extra characters from all columns in the DataFrame
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column is of string type
        df[column] = df[column].str.strip("'")  # Remove extra characters

# Filter relevant columns
df_filtered = df[['body', 'sentiment']].dropna()


In [6]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d4...,kabargayo.com,Kredit sepeda motor bisa terbayar jika Anda me...,https://www.kabargayo.com/2024/09/19/kredit-se...,"Jakarta, VIVA – Pembayaran kredit sepeda motor...",19/09/2024 22.32,id,19/09/2024 22.32,Aldi Hadad,https://i1.wp.com/thumb.viva.co.id/media/front...,NaN,positive,NaN,"Hari Pembayaran Berbayar atau Harcilnas 2024,P...",NaN,5250000,Adira,NaN,NaN
1,e1e3f8d68b58568e8217b7562d48de634fceb0d8371356...,viva.co.id,Kredit Motor Bisa Lunas Jika Bayar Cicilan Tep...,https://www.viva.co.id/otomotif/tips/1753596-k...,"Jakarta, VIVA – Cicilan kredit motor yang seri...",19/09/2024 22.30,id,19/09/2024 22.30,Krisna Wicaksono,https://thumb.viva.co.id/media/frontend/thumbs...,NaN,positive,NaN,"Harinya Cicilan Lunas,2024,PT Adira Dinamika M...",NaN,5250000,Adira,NaN,NaN
2,dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf5...,kabarmegapolitan.pikiran-rakyat.com,Adira Finance Umumkan Pemenang HARCILNAS 2024:...,https://kabarmegapolitan.pikiran-rakyat.com/bi...,KABARMEGAPOLITAN.com - PT Adira Dinamika Multi...,19/09/2024 21.45,id,19/09/2024 21.45,Yuliansyah,https://assets.pikiran-rakyat.com/www/network/...,NaN,positive,HARCILNAS merupakan wujud apresiasi kami kepad...,"PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 p...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b2...,banggairaya.id,"Dapatkan Promo Menarik, Yamaha Prima Motor Ram...",https://banggairaya.id/dapatkan-promo-menarik-...,BANGGAI RAYA- Yamaha Prima Motor ramaikan pame...,19/09/2024 19.45,id,19/09/2024 19.45,Chikal Connect,https://i0.wp.com/banggairaya.id/wp-content/up...,NaN,neutral,NaN,"RAYA- Yamaha Prima Motor,Banggai Goverment Exp...",NaN,5250000,Adira,NaN,NaN
4,1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721d...,jakarta.tribunnews.com,"Sindikat Penipuan Leasing, Satu Bulan Ajukan K...",https://jakarta.tribunnews.com/2024/09/19/sind...,Laporan wartawan TribunJakarta.com Yusuf Bacht...,19/09/2024 18.43,id,19/09/2024 18.43,Yusuf Bachtiar,https://asset-2.tstatic.net/jakarta/foto/bank/...,NaN,neutral,Pelaku ini melakukan pembiayaan pembelian kend...,"Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,T...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [4]:
# Balance dataset classes
class_counts = df_filtered['sentiment'].value_counts()
min_class = class_counts.min()
df_balanced = df_filtered.groupby('sentiment').apply(lambda x: x.sample(min_class)).reset_index(drop=True)

In [5]:
# Ensure balanced classes during split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_filtered['body'].tolist(), df_filtered['sentiment'].tolist(),
    test_size=0.2, random_state=42, stratify=df_filtered['sentiment']
)

In [9]:
from transformers import LongformerTokenizer

# Load tokenizer and tokenize texts
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


KeyboardInterrupt: 

In [6]:
# Map sentiment labels to integers
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
train_labels = [label_mapping[label] for label in train_labels]
val_labels = [label_mapping[label] for label in val_labels]

In [11]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [12]:
# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

NameError: name 'train_encodings' is not defined

In [15]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [16]:
# Load Longformer model for sequence classification
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Adjust model's loss function to include class weights
model.config.problem_type = "single_label_classification"
model.config.class_weights = class_weights.tolist()

In [18]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return {'eval_accuracy': acc}


In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    num_train_epochs=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    fp16=True,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',
    greater_is_better=True,
    lr_scheduler_type="cosine_with_restarts",
    warmup_steps=100,
    save_total_limit=2,
)


c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Include custom metrics
)


In [21]:
import torch

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [22]:
# Train the model
trainer.train()

  0%|          | 0/1800 [00:00<?, ?it/s]

Initializing global attention on CLS token...


{'loss': 1.0519, 'grad_norm': 8.462108612060547, 'learning_rate': 3e-05, 'epoch': 0.22}
{'loss': 0.9107, 'grad_norm': 10.316316604614258, 'learning_rate': 2.97596503983039e-05, 'epoch': 0.44}
{'loss': 0.7769, 'grad_norm': 9.78371524810791, 'learning_rate': 2.90169091223945e-05, 'epoch': 0.67}
{'loss': 0.7615, 'grad_norm': 13.819013595581055, 'learning_rate': 2.7796838817751467e-05, 'epoch': 0.89}


  0%|          | 0/57 [00:00<?, ?it/s]

{'eval_accuracy': 0.7162687395891172, 'eval_loss': 0.6535650491714478, 'eval_runtime': 206.0779, 'eval_samples_per_second': 8.739, 'eval_steps_per_second': 0.277, 'epoch': 1.0}
{'loss': 0.6619, 'grad_norm': 12.996597290039062, 'learning_rate': 2.6140987515286368e-05, 'epoch': 1.11}
{'loss': 0.6122, 'grad_norm': 9.344146728515625, 'learning_rate': 2.4127755718208092e-05, 'epoch': 1.33}
{'loss': 0.579, 'grad_norm': 11.549363136291504, 'learning_rate': 2.178514723892878e-05, 'epoch': 1.56}
{'loss': 0.572, 'grad_norm': 12.792920112609863, 'learning_rate': 1.9211478708314894e-05, 'epoch': 1.78}
{'loss': 0.5505, 'grad_norm': 16.03973960876465, 'learning_rate': 1.6494393321401312e-05, 'epoch': 2.0}


...

  0%|          | 0/57 [00:00<?, ?it/s]

{'eval_accuracy': 0.7784564131038312, 'eval_loss': 0.583918571472168, 'eval_runtime': 205.4234, 'eval_samples_per_second': 8.767, 'eval_steps_per_second': 0.277, 'epoch': 3.0}
{'loss': 0.3867, 'grad_norm': 17.520103454589844, 'learning_rate': 3.989867611983722e-06, 'epoch': 3.11}
{'loss': 0.3385, 'grad_norm': 17.60910415649414, 'learning_rate': 2.305461520582054e-06, 'epoch': 3.33}
{'loss': 0.3382, 'grad_norm': 14.383126258850098, 'learning_rate': 1.0533527116762298e-06, 'epoch': 3.56}
{'loss': 0.3162, 'grad_norm': 11.31326961517334, 'learning_rate': 2.7618024901454544e-07, 'epoch': 3.78}
{'loss': 0.3426, 'grad_norm': 15.031599998474121, 'learning_rate': 4.098086973186588e-10, 'epoch': 4.0}


  0%|          | 0/57 [00:00<?, ?it/s]

{'eval_accuracy': 0.7684619655746807, 'eval_loss': 0.6503986716270447, 'eval_runtime': 201.3352, 'eval_samples_per_second': 8.945, 'eval_steps_per_second': 0.283, 'epoch': 4.0}
{'train_runtime': 25024.3787, 'train_samples_per_second': 1.151, 'train_steps_per_second': 0.072, 'train_loss': 0.5550010999043783, 'epoch': 4.0}


TrainOutput(global_step=1800, training_loss=0.5550010999043783, metrics={'train_runtime': 25024.3787, 'train_samples_per_second': 1.151, 'train_steps_per_second': 0.072, 'total_flos': 9458731243929600.0, 'train_loss': 0.5550010999043783, 'epoch': 4.0})

In [23]:
print(f"Number of training examples: {len(train_dataset)}")
print(f"Batch size: {training_args.per_device_train_batch_size}")


Number of training examples: 7200
Batch size: 16


In [24]:
# Save the trained model and tokenizer
model.save_pretrained('./longformer_model7')
tokenizer.save_pretrained('./longformer_model7')

('./longformer_model7\\tokenizer_config.json',
 './longformer_model7\\special_tokens_map.json',
 './longformer_model7\\vocab.json',
 './longformer_model7\\merges.txt',
 './longformer_model7\\added_tokens.json')

In [25]:
# Evaluate the model
results = trainer.evaluate()

  0%|          | 0/57 [00:00<?, ?it/s]

In [26]:
print(results)

{'eval_accuracy': 0.7784564131038312, 'eval_loss': 0.583918571472168, 'eval_runtime': 200.6016, 'eval_samples_per_second': 8.978, 'eval_steps_per_second': 0.284, 'epoch': 4.0}


In [27]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# After training, get predictions on the validation dataset
predictions = trainer.predict(val_dataset)

# Get the predicted logits
logits = predictions.predictions

# Convert logits to predicted class labels
predicted_labels = np.argmax(logits, axis=1)

# Get the true labels
true_labels = val_labels

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


  0%|          | 0/57 [00:00<?, ?it/s]

Accuracy: 0.7785
F1 Score: 0.7788
Precision: 0.7794
Recall: 0.7785

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.72      0.70       338
           1       0.80      0.79      0.80       836
           2       0.80      0.80      0.80       627

    accuracy                           0.78      1801
   macro avg       0.76      0.77      0.76      1801
weighted avg       0.78      0.78      0.78      1801



In [28]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [29]:
# Load the saved model and tokenizer
model = LongformerForSequenceClassification.from_pretrained('./longformer_model6').to(device)
tokenizer = LongformerTokenizer.from_pretrained('./longformer_model6')

In [7]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model7'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!


Initializing global attention on CLS token...


Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobilitas yang baru dan segar," kata Fransiscu

: 